In [8]:
using JLD2
using Statistics
using DataFrames

In [9]:
algs = ["upwindPraveen"; "upwindClassic"; "upwindClassic"; "muscl"; "muscl"];
orders = [1; 1; 2; 1; 2];
totalNumGrids = 50;

In [10]:
function loadData(dataFolder)
    m1 = Matrix{Float64}(undef, 3, 5)
    N1 = Vector{Int64}(undef, 3)
    m2 = Matrix{Float64}(undef, 3, 5)
    N2 = Vector{Int64}(undef, 3)

    i1 = 1
    i2 = 1
    for file in readdir(dataFolder)
        d = load(dataFolder*file)
        alphas = d["alphas"]
        unstableSim = map(x -> x > 0.0 ? true : false, d["res"])
        unstableSims = mapslices(count, unstableSim, dims=1)
        if occursin("exp", file)
            m1[i1, :] .= 100*unstableSims[:]/totalNumGrids
            N1[i1] = d["N"]
            i1 += 1
        else
            m2[i2, :] .= 100*unstableSims[:]/totalNumGrids
            N2[i2] = d["N"]
            i2 += 1
        end
    end

    dfExp = DataFrame(m1, ["$(alg) $(order)" for (alg, order) in zip(algs, orders)]);
    dfExp[!, "N"] = N1;
    sort!(dfExp, :N);
    dfinverse = DataFrame(m2, ["$(alg) $(order)" for (alg, order) in zip(algs, orders)]);
    dfinverse[!, "N"] = N2;
    sort!(dfinverse, :N);
    return dfExp, dfinverse
end


loadData (generic function with 1 method)

The tables below contain the amount of times out of 100 that the schemes (2D) yielded unstable eigenvalues. The final column gives is the grid size. The top table is for an exponential weight function, the bottom table for an inverse weight function.

Noise for grids with particles chosen randomly in each cell [$-\Delta x/2$, $\Delta x/2$] ($r = \Delta x/2$). The top table is always for the exponential weight function, the second for the inverse weight function.

In [11]:
for folder in ["$(@__DIR__)/data2DNoise$(x)/" for x in range(1, 8)]
    dfExp, _ = loadData(folder);
    println(folder)
    println(dfExp)
end

/scratch/cac13ruw/meshfree4hypeq/numericalExperiments/gridSensitivity/data2DNoise1/
3×6 DataFrame
 Row │ upwindPraveen 1  upwindClassic 1  upwindClassic 2  muscl 1  muscl 2  N     
     │ Float64          Float64          Float64          Float64  Float64  Int64 
─────┼────────────────────────────────────────────────────────────────────────────
   1 │             0.0              0.0              0.0      0.0      0.0     30
   2 │             0.0              0.0              0.0      0.0      0.0     40
   3 │             0.0              0.0              0.0      0.0      0.0     50
/scratch/cac13ruw/meshfree4hypeq/numericalExperiments/gridSensitivity/data2DNoise2/
3×6 DataFrame
 Row │ upwindPraveen 1  upwindClassic 1  upwindClassic 2  muscl 1  muscl 2  N     
     │ Float64          Float64          Float64          Float64  Float64  Int64 
─────┼────────────────────────────────────────────────────────────────────────────
   1 │             0.0              0.0              0.0    

In [12]:
file = load("$(@__DIR__)/data2DNoise7/gridSensitivity2D_50_expWeight_alpha1.jld2")
eigs = file["eigs"]

for i = 1:length(algs)
    println("Alg: $(algs[i]), order: $(orders[i]), unstable eigenvalues: $([x for x in eigs[:, i] if x != 0.0])")
end

Alg: upwindPraveen, order: 1, unstable eigenvalues: Any[]
Alg: upwindClassic, order: 1, unstable eigenvalues: Any[]
Alg: upwindClassic, order: 2, unstable eigenvalues: Any[]
Alg: muscl, order: 1, unstable eigenvalues: Any[]
Alg: muscl, order: 2, unstable eigenvalues: Any[]
